In [1]:
import cv2
import numpy as np
from collections import deque
import tensorflow as tf  # Assuming you are using TensorFlow for the model

# Load your pre-trained model
model = tf.keras.models.load_model('model_8000.h5')

# Define classes_list (replace with your actual list of classes)
classes_list = ["Abuse","Arrest","Arson","Assault"]

# Define image dimensions
image_height = 64
image_width = 64

def save_clip(frames, output_file_path):
    if not frames:
        return

    height, width, _ = frames[0].shape
    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (width, height))

    for frame in frames:
        video_writer.write(frame)

    video_writer.release()

def predict_on_live_video(video_file_path, output_file_path, window_size):
    frame_count = 0
    frames = []
    predicted_label = 4
    predicted_labels_probabilities_deque = deque(maxlen=window_size)

    video_reader = cv2.VideoCapture(video_file_path)

    original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
    original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))

    video_writer = cv2.VideoWriter(output_file_path, cv2.VideoWriter_fourcc('M', 'P', '4', 'V'), 24, (original_video_width, original_video_height))

    while True:
        status, frame = video_reader.read()

        if not status:
            break  # This break statement is inside the while loop

        resized_frame = cv2.resize(frame, (image_height, image_width))
        normalized_frame = resized_frame / 255

        predicted_labels_probabilities = model.predict(np.expand_dims(normalized_frame, axis=0))[0]

        predicted_labels_probabilities_deque.append(predicted_labels_probabilities)

        if len(predicted_labels_probabilities_deque) == window_size:
            predicted_labels_probabilities_np = np.array(predicted_labels_probabilities_deque)
            predicted_labels_probabilities_averaged = predicted_labels_probabilities_np.mean(axis=0)

            predicted_max = np.max(predicted_labels_probabilities_averaged)
            predicted_label = np.argmax(predicted_labels_probabilities_averaged)

            if predicted_max > 0.6 and predicted_label < 4:
                frame_count += 1
                if frame_count < 450:
                    frames.append(frame)
                else:
                    if len(frames) > 100:
                        save_clip(frames, output_file_path)
                    frame_count = 0
                    frames = []
            else:
                predicted_label = 4

            if predicted_label < len(classes_list):  # Ensure predicted_label is within the valid range
                predicted_class_name = classes_list[predicted_label]
                cv2.putText(frame, predicted_class_name, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            else:
                print(f"Invalid predicted_label: {predicted_label}. Check classes_list length.")

        video_writer.write(frame)
        cv2.imshow('Predicted Frames', frame)

        key_pressed = cv2.waitKey(10)

        if key_pressed == ord('q'):
            break
            

    cv2.destroyAllWindows()
    video_reader.release()
    video_writer.release()

# Example usage
output_file_path = "C:/Users/ryzen/Downloads/VID-20240510-WA0004.mp4"
window_size = 30
video_file_path = "C:/Users/ryzen/seeya/video.mp4"
predict_on_live_video(video_file_path, output_file_path, window_size)


1/1 [==============================] - 0s 41ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 38ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 40ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 41ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 39ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 42ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 40ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 40ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] - 0s 44ms/step
Invalid predicted_label: 4. Check classes_list length.
1/1 [==============================] 

In [7]:
pip install opencv-python



SyntaxError: invalid syntax (556306433.py, line 1)

In [3]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_true, y_pred)


NameError: name 'y_true' is not defined

In [8]:
model = tf.keras.models.load_model('model_8000.h5')
print(model.summary())


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 64)        1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 60, 60, 64)        36928     
                                                                 
 batch_normalization (Batch  (None, 60, 60, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 30, 30, 64)        0         
 D)                                                              
                                                                 
 global_average_pooling2d (  (None, 64)                0         
 GlobalAveragePooling2D)                                         
                                                        

In [ ]:

#1. Convolutional layer (`Conv2D`) with 64 filters and a kernel size of 3x3, resulting in an output shape of (None, 62, 62, 64).
#2. Convolutional layer (`Conv2D`) with 64 filters and a kernel size of 3x3, resulting in an output shape of (None, 60, 60, 64).
#3. Batch normalization layer (`BatchNormalization`) with an output shape of (None, 60, 60, 64).
#4. Max pooling layer (`MaxPooling2D`) with a pool size of 2x2, resulting in an output shape of (None, 30, 30, 64).
#5. Global average pooling layer (`GlobalAveragePooling2D`) with an output shape of (None, 64).
#6. Dense layer (`Dense`) with 256 units, resulting in an output shape of (None, 256).
#7. Batch normalization layer (`BatchNormalization`) with an output shape of (None, 256).
#8. Dense layer (`Dense`) with 4 units, representing the output classes.

#So, there are a total of **7 layers** in the model architecture.

In [ ]:
#1. Local Connectivity: CNNs use convolutional layers that apply filters (kernels) to local regions of the image, 
# capturing local patterns such as edges, textures, and simple shapes. 

#2. Shared Weights: The same filter is applied across different parts of the image, which allows the network to learn translation-invariant features. 
# This means the network can recognize patterns regardless of their position in the image.

#3. Hierarchical Feature Learning: CNNs build hierarchical feature representations. Lower layers capture simple features (edges, textures),
#  while deeper layers combine these features to recognize more complex patterns (object parts, objects). 